#### URL 이미지 불러오기

In [1]:
import requests
from PIL import Image, ImageDraw
from io import BytesIO
import gradio as gr


IMAGE_URL = "https://cdn.pixabay.com/photo/2023/12/13/22/29/young-woman-8447841_1280.jpg"
ENDPOINT_URL = "https://fimtrus-vision2.cognitiveservices.azure.com/computervision/imageanalysis:analyze"
API_KEY = "1Xvo96LQT7LFAe6FgQUeO3f8UwOfELLcvZNJThBQW6LJsBdUhwzrJQQJ99BGACYeBjFXJ3w3AAAFACOGxEEM"


def random_color():
    import random
    # 랜덤한 RGB 색상 튜플 반환
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 

def get_font():
    # OS별로 적절한 폰트 객체 반환 (한글 지원)
    from PIL import ImageFont
    import platform
    
    font_size = 20
    
    try:
        if platform.system() == "Windows":
            # 윈도우용 한글 폰트
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":  # macOS
            # 맥용 한글 폰트
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else:  # Linux      
            # 리눅스 기본 폰트
            return ImageFont.load_default(size=font_size)
    except IOError:
        # 폰트 파일이 없을 경우 기본 폰트 사용
        return ImageFont.load_default(size=font_size)
    

def request_image_analysis(image_url, features=["objects"], **kwargs):
    endpoint = "{}".format(ENDPOINT_URL)
    
    params = {
        "api-version": "2024-02-01",
        "features": ",".join(features)
    }
    
    headers = {
        "Ocp-Apim-Subscription-Key": API_KEY
    }
    
    body = {
        "url": image_url
    }
    
    if kwargs is not None:
    
        if ("caption" in features or "denseCaptions" in features) and "gender_neutral_caption" in kwargs.keys() :
            params.update({
                "gender-neutral-caption": kwargs['gender_neutral_caption']
            })
            
        if "smartCrops" in features and "smartcrops_aspect_ratios" in kwargs.keys():
            params.update({
                "smartcrops-aspect-ratios": kwargs["smartcrops_aspect_ratios"]
            })
        
    response = requests.post(endpoint, params=params, headers=headers, json=body)
    
    if response.status_code != 200:
        return None
    
    response_json = response.json()
    
    return response_json
    
    

def draw_image(image_url, features, data):
    # 이미지 다운로드
    image_response = requests.get(image_url)
    image = Image.open(BytesIO(image_response.content))
    draw = ImageDraw.Draw(image)
    font = get_font()       # 폰트 객체

    for feature in features:
        
        if feature == "tags" or feature == "caption":
            continue
        
        feature_key = '{}Result'.format(feature)
        # 객체 인식 결과 리스트
        result_object = data[feature_key]
        block_list = result_object['values']
        
        color = random_color()  # 랜덤 색상
        
        for block in block_list:
            
            bounding_box = block['boundingBox']
            x, y, w, h = bounding_box['x'], bounding_box['y'], bounding_box['w'], bounding_box['h']
            
            if feature == "objects":
                tag = block['tags'][0]
                name = tag['name']
                confidence = tag['confidence']
        
                formatted_text = "{} ({:.2f}%)".format(name, confidence * 100)
                
            elif feature == "denseCaptions":
                formatted_text = block["text"]
            else:
                formatted_text = None
            
            # 객체 위치에 사각형과 라벨 그리기
            draw.rectangle([(x, y), (x + w, y + h)], outline=color, width=2)
            
            # Feature의 텍스트를 그려주는 부분
            feature_text_bbox = draw.textbbox((x, y), feature, font=font)            
            draw.rectangle([(feature_text_bbox[0], feature_text_bbox[1]),(feature_text_bbox[2], feature_text_bbox[3])], outline=color, fill=color, width=2)
            draw.text((x, y), feature, fill="black", font=font)
            
            # 텍스트를 그려주는 부분
            if formatted_text:
                text_bbox = draw.textbbox((x, y + 20), formatted_text, font=font)      
                draw.rectangle([(text_bbox[0], text_bbox[1]),(text_bbox[2] + 5, text_bbox[3])], outline=color, fill=color, width=2)
                draw.text((x + 5, y + 20), formatted_text, fill="white", font=font)
    
    return image


def get_result_text(features, response_data):
    result_text_list = []
        
    if "caption" in features:
        result_text_list.append("[Caption]")
        result_text_list.append(response_data['captionResult']['text'])
        result_text_list.append("{:.2f}%".format(response_data['captionResult']['confidence'] * 100))
        
    if "tags" in features:            
        if len(result_text_list) > 0:
            result_text_list.append("\n")
            
        result_text_list.append("[Tags]")
        tag_list = response_data['tagsResult']['values']
        for tag in tag_list:
            name = tag['name']
            confidence = tag['confidence']
            result_text_list.append("{}({:.2f}%)".format(name, confidence * 100))
            
    result_text = "\n".join(result_text_list)
    
    return result_text

theme = gr.themes.Soft(primary_hue=gr.themes.colors.indigo)
with gr.Blocks(theme=theme) as demo:
    
    FEATURES = ["objects", "caption", "denseCaptions", "tags", "smartCrops"]
    
    def click_send(image_url, features, is_gender_neutral, ratio):
        
        option = {}
        
        if "caption" in features or "denseCaptions" in features:
            option.update({"gender_neutral_caption": is_gender_neutral})
            
        if "smartCrops" in features:
            option.update({"smartcrops_aspect_ratios": ratio})
        
        response_data = request_image_analysis(image_url, features, **option)
        image = draw_image(image_url, features, response_data)
        result_text = get_result_text(features, response_data)
        
        return image, result_text
    
    def change_features(features):
        selected_caption = False
        selected_smart_crops = False
        
        if "caption" in features or "denseCaptions" in features:
            selected_caption = True
            
        if "smartCrops" in features:
            selected_smart_crops = True
        
        return features, gr.update(visible=selected_caption, interactive=True), gr.update(visible=selected_smart_crops, interactive=True)

    features_checkbox = gr.CheckboxGroup(label="Features", choices=FEATURES)
    # gender_neutral_caption
    gender_radio = gr.Radio(label="성중립성", choices=[("중립", True), ("구분", False)], value=False, visible=False)
    # smartcrops_aspect_ratios
    ratio_textbox = gr.Textbox(label="smartCrops 크기", placeholder="ex) 0.75,1.2,1.5", visible=False)
    image_url_textbox = gr.Textbox(label="이미지 URL")
    send_button = gr.Button("전송")
    
    with gr.Row():
        output_image = gr.Image(scale=3, label="결과 화면", interactive=False, type="pil")
        output_textbox = gr.TextArea(scale=1, label="결과 텍스트", lines=20)
    
    send_button.click(click_send, inputs=[image_url_textbox, features_checkbox, gender_radio, ratio_textbox], outputs=[output_image, output_textbox])
    features_checkbox.change(change_features, inputs=[features_checkbox], outputs=[features_checkbox, gender_radio, ratio_textbox])

    
demo.launch()



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
